In [3]:
import ipywidgets as wg
from ipywidgets import Layout
from IPython.display import display

vb = wg.VBox([wg.Text('1'),wg.Text('2')])
btn = wg.Button(description = 'Add') 

def on_bttn_clicked(b):        
    vb.children=tuple(list(vb.children) + [wg.Text('3')]) 

btn.on_click(on_bttn_clicked)
display(vb, btn)

list(vb.children)


import itk
from itkwidgets import view, compare, checkerboard
from itkwidgets.widget_viewer import Viewer
import ipywidgets
from ipywidgets import Button, Label, HBox, VBox
import os

# viewerFixed = Viewer(image=None)
# viewerMoving = Viewer(image=None)
top_box=HBox([Label('Fixed image'), Label('Moving image')])
# in_box=HBox([viewerFixed, viewerMoving])
mid_box=HBox([Label('Checkerboard before'), Label('Checkerboard after')])
# out_box=HBox([viewerFixed, viewerMoving])
imageViews = VBox([top_box, mid_box])

display(imageViews)

# these FileUpload widgets have maximum size
# controlled via jupyter_notebook_config.py
# https://github.com/jupyter-widgets/ipywidgets/issues/2522
fixedUploader = ipywidgets.FileUpload(
    accept='image/*,.nrrd,.mha,.nii',
    multiple=False
)
display(fixedUploader)

movingUploader = ipywidgets.FileUpload(
    accept='image/*,.nrrd,.mha,.nii',
    multiple=False
)
display(movingUploader)

viewButton = ipywidgets.Button(description="Load and view")
display(viewButton)

appState={'fixed':None, 'moving':None}
out = ipywidgets.Output(layout={'border': '2px solid blue'})
display(out)

def viewImages(b):
    movingButton=Button(description='Moving Updated')
    
    # write to temporary image file
    # I don't know how to read image from memory in Python
    # using FileUpload's syntax for ipywidgets <= 7
    tempFilename="./fixed_"+ next(iter(fixedUploader.value))
    with open(tempFilename, "wb") as fp:
        fp.write(fixedUploader.data[0])
    appState['fixed'] = itk.imread(tempFilename, itk.F)
    os.remove(tempFilename)
    # viewerFixed.image=appState['fixed']

    tempFilename="./moving_"+ next(iter(movingUploader.value))
    with open(tempFilename, "wb") as fp:
        fp.write(movingUploader.data[0])
    appState['moving'] = itk.imread(tempFilename, itk.F)
    os.remove(tempFilename)
    
    out.clear_output(wait=True)
    # with out:
    compareWidget=compare(appState['fixed'],
                          appState['moving'],
                          link_cmap=True,
                          link_gradient_opacity=True)
    imageViews.children=tuple(list(imageViews.children) + [compareWidget])
    #display(compareWidget)

viewButton.on_click(viewImages)

checkerboardButton = ipywidgets.Button(description="View checkerboard")
display(checkerboardButton)

def checkerboardImages(b):  
    display(checkerboard(appState['fixed'], appState['moving'], ui_collapsed=False))

checkerboardButton.on_click(checkerboardImages)

regType=ipywidgets.RadioButtons(
    options=['rigid', 'affine', 'bspline'],
    value='rigid', # Defaults to 'pineapple'
    description='Type:',
    disabled=False
)
display(regType)

registerButton = ipywidgets.Button(description="Register and view")
display(registerButton)


def registerImages(b):
    out.clear_output(wait=True)
    parameters = itk.ParameterObject.New()

    resolutions = 3
    default_rigid = parameters.GetDefaultParameterMap("rigid", resolutions)
    parameters.AddParameterMap(default_rigid)

    if (regType.value!='rigid'):
        resolutions = 2
        default_affine = parameters.GetDefaultParameterMap("affine", resolutions)
        parameters.AddParameterMap(default_affine)

    if (regType.value=='bspline'):
        resolutions = 1
        default_bspline = parameters.GetDefaultParameterMap("bspline", resolutions)
        parameters.AddParameterMap(default_bspline)

    parameters.RemoveParameter("ResultImageFormat")
    with out:
        print('Executing '+regType.value+' registration. Please wait...')
    
    registered_moving = itk.elastix_registration_method(appState['fixed'], appState['moving'], parameter_object=parameters)
    display(checkerboard(appState['fixed'], registered_moving))
    out.clear_output()

registerButton.on_click(registerImages)

Button(description='Add', style=ButtonStyle())

FileUpload(value={}, accept='image/*,.nrrd,.mha,.nii', description='Upload')

FileUpload(value={}, accept='image/*,.nrrd,.mha,.nii', description='Upload')

Button(description='Load and view', style=ButtonStyle())

Output(layout=Layout(border='2px solid blue'))

Button(description='View checkerboard', style=ButtonStyle())

RadioButtons(description='Type:', options=('rigid', 'affine', 'bspline'), value='rigid')

Button(description='Register and view', style=ButtonStyle())